In [1]:
import pickle as pkl
import numpy as np
import keras
import boto3
import numpy as np
import pandas as pd
import os
import json
from datetime import timedelta
from dateutil import parser
import boto3
import urllib.request
import datetime
import sklearn
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
%cd "D:\fresh_air\april"

D:\fresh_air\april


In [7]:
X = pkl.load(open("x_master.p", "rb"))
Y = pkl.load(open("y_master.p", "rb"))
x_test1911 = pkl.load(open("x_test1911.p", "rb"))
y_test1911 = pkl.load(open("y_test1911.p", "rb"))

In [8]:
x_bin=[]
y_bin=[]
smoking_event=0
non_smoking_event=0


for i, y in enumerate(Y):
  if X[i].shape == (160, 4):
      if y in [1,2,3]:
        y_bin.append(1)
        x_bin.append(X[i])
        smoking_event+=1
      elif y >= 20:
        y_bin.append(0)
        x_bin.append(X[i])
        non_smoking_event+=1
        
x_bin=np.array(x_bin)
y_bin=np.array(y_bin)

In [9]:
x_test1911_bin=[]
y_test1911_bin=[]
smoking_event=0
non_smoking_event=0


for i, y in enumerate(y_test1911):
  if x_test1911[i].shape == (160, 4):
      if y in [1,2,3]:
        y_test1911_bin.append(1)
        x_test1911_bin.append(x_test1911[i])
        smoking_event+=1
      elif y >= 20:
        y_test1911_bin.append(0)
        x_test1911_bin.append(x_test1911[i])
        non_smoking_event+=1
        
x_test1911_bin=np.array(x_test1911_bin)
y_test1911_bin=np.array(y_test1911_bin)

In [10]:
print(x_bin.shape)
print(y_bin.shape)
print(x_test1911_bin.shape)
print(y_test1911_bin.shape)

(83369, 160, 4)
(83369,)
(10908, 160, 4)
(10908,)


In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv1D,Conv2D, MaxPooling1D
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

In [13]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(x_bin.shape[1],x_bin.shape[2])))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())

model.add(Dense(128,activation='relu' ))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, kernel_initializer = 'glorot_normal', activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])

history = model.fit(x_bin, y_bin, validation_data=(x_test1911_bin, y_test1911_bin), epochs=300,verbose=0)
scores_ = model.evaluate(x_test1911_bin, y_test1911_bin)
print("%s: %.2f%%" % (model.metrics_names[1], scores_[1]*100))

10908/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

accuracy: 95.54%


In [16]:

pred_y  = model.predict_classes(x_test1911_bin)
print(classification_report(y_test1911_bin,pred_y))

              precision    recall  f1-score   support

           0       0.97      0.98      0.98     10188
           1       0.70      0.58      0.63       720

    accuracy                           0.96     10908
   macro avg       0.83      0.78      0.80     10908
weighted avg       0.95      0.96      0.95     10908



In [18]:
model.save("Model2_VGG_FreshAir.h5")

In [15]:
pred_y

array([[1],
       [1],
       [0],
       ...,
       [0],
       [0],
       [0]])